# With this notebook you can quantitatively evaluate your deep learning performance

In [15]:
import os, fnmatch
import SimpleITK as sitk
import numpy as np
import scipy as sp
from scipy import ndimage as nd
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Computer Modern Roman"
import matplotlib as mpl
from matplotlib import ticker, cm
from skimage.measure import compare_ssim as ssim
from scipy.stats import pearsonr
import copy
from matplotlib.colorbar import Colorbar
from scipy.spatial.distance import jaccard
from sklearn.metrics import adjusted_rand_score as rand
from sklearn.metrics import mutual_info_score as infoscore
from sklearn.metrics import roc_curve, auc

In [16]:
root_dir = "/home/dominik/ICBDeepLearningPipeline/data/Ouncomol_NucEnvelope_3Dnew/"
#root_dir = "/media/dominik/FertExperim/ScottDatasetsize/Scott100Percent/"
savepath = root_dir + "insights/"
max_images = 200


In [3]:
%run "/home/dominik/ICBDeepLearningPipeline/Preprocessing_Evaluation/Utils_data_evaluation.py"
prepare_data_for_evaluation(root_dir, max_images)

Checking dimensions:
Shape of data imported is: (40, 16, 128, 128, 3)
Checking dimensions:
Shape of data imported is: (40, 16, 128, 128, 3)
Checking dimensions:
Shape of data imported is: (40, 16, 128, 128, 3)
Checking dimensions:
Shape of data imported is: (40, 16, 128, 128, 3)
(40, 16, 128, 128, 3)
(40, 16, 128, 128, 3)
The relative Error over the normalized dataset is: 1.1065398247383909  best ist close to zero


In [4]:
prediction = np.load(root_dir + "/insights/" + "prediction.npy")
groundtruth = np.load(root_dir + "/insights/" + "groundtruth.npy")
abs_errormap_norm = np.load(root_dir + "/insights/" + "abs_errormap.npy")
rel_errormap_norm = np.load(root_dir + "/insights/" + "rel_errormap.npy")

print(np.shape(prediction))
print(np.shape(groundtruth))

(40, 16, 128, 128, 3)
(40, 16, 128, 128, 3)


In [5]:
›

In [6]:
from skimage.color import rgb2gray

prediction_binary = copy.deepcopy(prediction)
groundtruth_binary = copy.deepcopy(groundtruth)
pred_mean = np.mean(prediction_binary)
gt_mean = np.mean(groundtruth_binary)

prediction_binary[prediction_binary <= pred_mean] = 0
prediction_binary[prediction_binary > pred_mean] = 1
groundtruth_binary[groundtruth_binary <= gt_mean] = 0
groundtruth_binary[groundtruth_binary > gt_mean] = 1

In [7]:
print(np.min(prediction_binary))
print(np.mean(prediction_binary))
print(np.max(prediction_binary))

print(np.min(groundtruth_binary))
print(np.mean(groundtruth_binary))
print(np.max(groundtruth_binary))

0.0
0.2893509
1.0
0
0.3310696601867676
1


In [8]:
def corrcoeff(a,b): 
    mean_a = np.mean(a)
    mean_b = np.mean(b)
    std_a = np.std(a)
    std_b = np.std(b)
    return np.mean((a - mean_a)*(b-mean_b))/(std_a*std_b)

In [9]:
Pearson, Pearson_all = getPearson(prediction, groundtruth)
print(Pearson)
print(corrcoeff(prediction, groundtruth))
data_to_plotPearson = [np.array(Pearson_all)]#[np.array(PearsonGFP), np.array(PearsonBF)] #[np.array(PearsonBoth), np.array(PearsonGFP), np.array(PearsonBF)]

-0.020129265078551492
-0.01802567758360285


In [10]:
print(Pearson_all)

[ 9.41866433e-02 -1.27455535e-01  9.03823299e-02 -1.22469547e-01
 -7.44693655e-02  5.83965586e-02 -1.05554948e-02 -2.03406991e-02
  3.57643787e-02  3.87586077e-02 -7.31305716e-06  4.81161949e-03
  5.98606274e-02 -1.22799262e-01  1.85597380e-01 -1.01251178e-01
 -1.25083371e-01 -2.62967368e-02 -3.43777012e-02  8.67629652e-03
 -1.99178310e-02 -5.82444517e-02 -1.16895154e-01 -9.50382916e-02
 -4.37428503e-02 -5.10651458e-02 -2.49528414e-02  1.04318936e-01
 -1.01064869e-02 -4.73614554e-02 -1.01106173e-01  5.11659218e-03
  6.34117486e-02  1.38340452e-02 -1.21473775e-01 -4.34497422e-03
  2.38848822e-02 -1.15909056e-02 -3.96787852e-02 -3.54846221e-02]


In [11]:
groundtruth_norm = (groundtruth - np.min(groundtruth))/(np.max(groundtruth)-np.min(groundtruth))
prediction_norm = (groundtruth - np.min(prediction))/(np.max(prediction)-np.min(prediction))
rel_errormap_norm = np.abs(np.divide(abs_errormap_norm, groundtruth_norm, out=np.zeros_like(abs_errormap_norm), where=groundtruth_norm!=0))

In [12]:
abs_errormap_norm = (np.abs((prediction_binary-groundtruth_binary))/np.size(groundtruth_binary))
print(np.mean(abs_errormap_norm))
print(1-np.abs(rand(prediction_binary.flatten(), groundtruth_binary.flatten())))
print(infoscore(prediction_binary.flatten(), groundtruth_binary.flatten()))

1.3810861977011218e-08
0.9968617707165827
8.702595129486679e-05


In [13]:
#ROC AUC
from sklearn.metrics import roc_curve, auc # roc curve tools
groundtruth2 = (groundtruth-np.min(groundtruth))/(np.max(groundtruth)-np.min(groundtruth))
prediction2 = (prediction-np.min(prediction))/(np.max(prediction)-np.min(prediction))

ground_truth_labels = (groundtruth2.flatten()).astype(int) # we want to make them into vectors
print(np.mean(ground_truth_labels),np.max(ground_truth_labels))
print(np.mean(prediction2),np.max(prediction2))

score_value = prediction2.flatten()# we want to make them into vectors
fpr, tpr, _ = roc_curve(ground_truth_labels,score_value)
roc_auc = auc(fpr,tpr)
print(roc_auc)

3.814697265625e-06 1
0.3347491 1.0
0.5064255935691587


In [14]:
from sklearn.metrics import mutual_info_score
from sklearn.metrics import accuracy_score, adjusted_rand_score, auc, roc_auc_score

print(np.size(groundtruth))
print(np.shape(groundtruth))
print("Metriccs:")
print("The accuracy on the normalized dataset is: ", 1 - np.mean(np.square(groundtruth_norm-prediction_norm))/(groundtruth_norm.size))
print("The median relative error on the normalized dataset is: ", np.median(rel_errormap_norm)*100 , "%")
print("The mean absolute error on the normalized dataset is: ", np.mean(abs_errormap_norm))
print("The Pearson coefficient is: ", np.median(Pearson))
print("The Jaccard index is: ", jaccard(prediction_binary.flatten(), groundtruth_binary.flatten()))
print("The AUC is:", roc_auc_score(prediction_binary.flatten(), groundtruth_binary.flatten()))
#print("The Information score is: ", mutual_info_score(np.concatenate(np.concatenate(prediction_norm)), np.concatenate(np.concatenate(groundtruth_norm))))
#print("The rand score is:" , adjusted_rand_score(np.concatenate(np.concatenate(groundtruth_norm)), np.concatenate(np.concatenate(prediction_norm))))
f = open(savepath + '/Error analysis.txt','w')
f.write('\n' + "The median relative error on the normalized dataset is: " + str(np.median(rel_errormap_norm)*100) + " Prozent")
f.write('\n' + "The mean absolute error on the normalized dataset is: " + str(np.mean(abs_errormap_norm)))
f.write('\n' + "The Pearson coefficient is: " + str(Pearson))
f.close()

31457280
(40, 16, 128, 128, 3)
Metriccs:
The accuracy on the normalized dataset is:  0.9999999999999958
The median relative error on the normalized dataset is:  364.3133166520712 %
The mean absolute error on the normalized dataset is:  1.3810861977011218e-08
The Pearson coefficient is:  -0.020129265078551492
The Jaccard index is:  0.8237053642381591
The AUC is: 0.493164559335009
